# 📊 Step 4: 比較とまとめ

これまでの3つのノートブックで学んだ内容を統合し、**データ管理**、**Script Mode**、**Training Jobs**の違いと使い分けを理解します。

## 🎯 このノートブックで学ぶこと
- 3つのアプローチの詳細比較
- 実際のプロジェクトでの使い分け
- ベストプラクティスの理解
- 次のステップの提案

## ⏱️ 実行時間: 約3分

## 1. 実行結果の読み込みと確認

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

print("📊 実行結果の読み込み中...")

# 各ノートブックの実行結果を読み込み
try:
    with open('script_mode_info.json', 'r') as f:
        script_info = json.load(f)
    print("✅ Script Mode結果を読み込み")
except FileNotFoundError:
    print("❌ Script Mode結果が見つかりません")
    script_info = None

try:
    with open('training_jobs_info.json', 'r') as f:
        training_jobs_info = json.load(f)
    print("✅ Training Jobs結果を読み込み")
except FileNotFoundError:
    print("❌ Training Jobs結果が見つかりません")
    training_jobs_info = None

try:
    with open('s3_data_paths.json', 'r') as f:
        s3_paths = json.load(f)
    print("✅ S3データパス情報を読み込み")
except FileNotFoundError:
    print("❌ S3データパス情報が見つかりません")
    s3_paths = None

print(f"\n📋 読み込み状況:")
print(f"  💻 Script Mode: {'✅' if script_info else '❌'}")
print(f"  ☁️ Training Jobs: {'✅' if training_jobs_info else '❌'}")
print(f"  📊 S3データ: {'✅' if s3_paths else '❌'}")

## 2. 詳細比較表の作成

In [ ]:
print("📊 詳細比較表を作成中...")

# 比較データの準備
if script_info and training_jobs_info:
    comparison_data = {
        '項目': [
            '実行時間 (秒)',
            'データソース',
            '実行環境',
            'デバッグの容易さ',
            'スケーラビリティ',
            '概算コスト',
            '再現性',
            '本番適用性',
            'テスト精度'
        ],
        'Script Mode': [
            f"{script_info['total_time']:.1f}",
            'S3から直接読み込み',
            'ノートブック内',
            '高（即座に確認）',
            '低（インスタンス制限）',
            'ノートブック料金のみ',
            '中（環境依存）',
            '低（実験向け）',
            f"{script_info['test_accuracy']:.4f}"
        ],
        'Training Jobs': [
            f"{training_jobs_info['total_time']:.1f}",
            'S3から自動取得',
            'ml.m5.large',
            '中（ログで確認）',
            '高（複数インスタンス対応）',
            f"${training_jobs_info['actual_cost']:.4f}",
            '高（コンテナ環境）',
            '高（本番向け）',
            'CloudWatchログで確認'
        ]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n=== 📊 詳細比較結果 ===")
    print(comparison_df.to_string(index=False))
    
else:
    print("⚠️ 比較に必要なデータが不足しています")
    print("🔄 先に 02_script_mode_training_complete.ipynb と 03_training_jobs.ipynb を実行してください")

## 3. 視覚的な比較

In [ ]:
if script_info and training_jobs_info:
    # 比較グラフの作成
    plt.figure(figsize=(15, 10))
    
    # 実行時間比較
    plt.subplot(2, 3, 1)
    methods = ['Script Mode', 'Training Jobs']
    times = [script_info['total_time'], training_jobs_info['total_time']]
    colors = ['lightblue', 'lightcoral']
    bars = plt.bar(methods, times, color=colors)
    plt.title('実行時間比較')
    plt.ylabel('時間（秒）')
    for bar, time_val in zip(bars, times):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                 f'{time_val:.1f}s', ha='center', va='bottom')
    
    # コスト比較
    plt.subplot(2, 3, 2)
    costs = [0, training_jobs_info['actual_cost']]  # Script Modeは追加コストなし
    bars = plt.bar(methods, costs, color=colors)
    plt.title('コスト比較')
    plt.ylabel('コスト（USD）')
    for bar, cost in zip(bars, costs):
        if cost > 0:
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0001, 
                     f'${cost:.4f}', ha='center', va='bottom')
        else:
            plt.text(bar.get_x() + bar.get_width()/2, 0.0005, 
                     '追加コストなし', ha='center', va='bottom')
    
    # 特徴比較レーダーチャート風
    plt.subplot(2, 3, 3)
    categories = ['実行速度', 'デバッグ性', 'スケール性', '本番適用']
    script_scores = [4, 5, 2, 2]  # Script Modeのスコア
    training_scores = [3, 3, 5, 5]  # Training Jobsのスコア
    
    x = np.arange(len(categories))
    width = 0.35
    
    plt.bar(x - width/2, script_scores, width, label='Script Mode', color='lightblue')
    plt.bar(x + width/2, training_scores, width, label='Training Jobs', color='lightcoral')
    
    plt.xlabel('特徴')
    plt.ylabel('スコア (1-5)')
    plt.title('特徴別比較')
    plt.xticks(x, categories, rotation=45)
    plt.legend()
    plt.ylim(0, 6)
    
    # データ管理フローの図解
    plt.subplot(2, 3, 4)
    plt.text(0.5, 0.8, '📊 データ管理フロー', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=14, weight='bold')
    plt.text(0.5, 0.6, '1. ローカルデータ作成', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=10)
    plt.text(0.5, 0.5, '↓', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=12)
    plt.text(0.5, 0.4, '2. S3アップロード', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=10)
    plt.text(0.5, 0.3, '↓', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=12)
    plt.text(0.5, 0.2, '3. 機械学習実行', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=10)
    plt.axis('off')
    
    # 学習した技術スタック
    plt.subplot(2, 3, 5)
    plt.text(0.5, 0.9, '🎓 学習した技術', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=14, weight='bold')
    skills = ['S3データ管理', 'pandas読み込み', 'scikit-learn', 'SageMaker SDK']
    for i, skill in enumerate(skills):
        plt.text(0.5, 0.7 - i*0.15, f'✅ {skill}', ha='center', va='center', 
                 transform=plt.gca().transAxes, fontsize=10)
    plt.axis('off')
    
    # 次のステップ
    plt.subplot(2, 3, 6)
    plt.text(0.5, 0.9, '🚀 次のステップ', ha='center', va='center', 
             transform=plt.gca().transAxes, fontsize=14, weight='bold')
    next_steps = ['大規模データ', 'MLOpsパイプライン', '自動化', '本番デプロイ']
    for i, step in enumerate(next_steps):
        plt.text(0.5, 0.7 - i*0.15, f'📈 {step}', ha='center', va='center', 
                 transform=plt.gca().transAxes, fontsize=10)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 視覚的比較完了")
else:
    print("⚠️ グラフ作成に必要なデータが不足しています")

## 4. 実際のプロジェクトでの使い分け指針

In [ ]:
print("🎯 実際のプロジェクトでの使い分け指針:")

use_cases = {
    "📊 データ管理": {
        "いつ使う": "すべてのプロジェクトで必須",
        "重要ポイント": [
            "構造化された命名規則",
            "バージョン管理",
            "データ品質チェック",
            "アクセス制御"
        ],
        "ベストプラクティス": "s3://bucket/project/data/version/type/"
    },
    "💻 Script Mode": {
        "いつ使う": "探索・実験・プロトタイプ段階",
        "適用場面": [
            "データ探索・可視化",
            "特徴量エンジニアリング",
            "モデル選択・比較",
            "ハイパーパラメータ初期調整"
        ],
        "メリット": "高速な実験サイクル、即座のデバッグ"
    },
    "☁️ Training Jobs": {
        "いつ使う": "本格開発・本番運用段階",
        "適用場面": [
            "最終モデルの訓練",
            "大規模データ処理",
            "分散訓練",
            "自動化パイプライン"
        ],
        "メリット": "スケーラビリティ、再現性、本番準備"
    }
}

for approach, details in use_cases.items():
    print(f"\n{approach}")
    print(f"  🎯 {details['いつ使う']}")
    
    if '適用場面' in details:
        print(f"  📋 適用場面:")
        for scene in details['適用場面']:
            print(f"    • {scene}")
    
    if '重要ポイント' in details:
        print(f"  📋 重要ポイント:")
        for point in details['重要ポイント']:
            print(f"    • {point}")
    
    if 'メリット' in details:
        print(f"  ✨ メリット: {details['メリット']}")
    
    if 'ベストプラクティス' in details:
        print(f"  💡 ベストプラクティス: {details['ベストプラクティス']}")

print(f"\n🔄 推奨開発フロー:")
workflow = [
    "1. 📊 データ管理: S3への構造化保存",
    "2. 💻 Script Mode: 迅速な実験・プロトタイプ",
    "3. ☁️ Training Jobs: 本格訓練・本番準備",
    "4. 🚀 デプロイ: エンドポイント・バッチ推論",
    "5. 📈 監視: パフォーマンス・ドリフト検出"
]

for step in workflow:
    print(f"  {step}")

## 5. 学習成果のまとめ

In [ ]:
print("🎓 学習成果のまとめ:")

learning_outcomes = {
    "📊 データ管理スキル": [
        "S3での構造化データ保存",
        "データのバージョン管理",
        "品質チェックと整合性確認",
        "効率的なアップロード・ダウンロード"
    ],
    "💻 Script Mode習得": [
        "ノートブック内での機械学習パイプライン",
        "複数モデルの比較・評価",
        "ハイパーパラメータ最適化",
        "インタラクティブなデバッグ"
    ],
    "☁️ Training Jobs理解": [
        "スケーラブルなクラウド訓練",
        "自動的なリソース管理",
        "本番環境への準備",
        "コスト意識とモニタリング"
    ],
    "🔧 実践的スキル": [
        "適切な手法の選択基準",
        "効率的な開発フロー",
        "リソースの最適化",
        "実際のプロジェクト適用"
    ]
}

total_skills = 0
for category, skills in learning_outcomes.items():
    print(f"\n{category}")
    for skill in skills:
        print(f"  ✅ {skill}")
        total_skills += 1

print(f"\n🎉 総習得スキル数: {total_skills}個")

# 実行統計のサマリー
if script_info and training_jobs_info:
    print(f"\n📊 今回の実行統計:")
    print(f"  ⏱️ Script Mode実行時間: {script_info['total_time']:.1f}秒")
    print(f"  ⏱️ Training Jobs実行時間: {training_jobs_info['total_time']:.1f}秒")
    print(f"  💰 Training Jobsコスト: ${training_jobs_info['actual_cost']:.4f}")
    print(f"  🎯 Script Mode精度: {script_info['test_accuracy']:.4f}")
    print(f"  📊 使用データ: 3,000サンプル、30特徴量")

print(f"\n🚀 次のステップ提案:")
next_steps = [
    "🔍 より大きなデータセットでの実験",
    "🧠 深層学習モデルの試行",
    "🔄 MLOpsパイプラインの構築",
    "📈 A/Bテストとモデル監視",
    "🌐 実際のビジネス問題への適用"
]

for step in next_steps:
    print(f"  {step}")

print(f"\n🎓 講義完了！機械学習の実践的なスキルを習得しました！")
print(f"📚 学んだ知識を活用して、実際のプロジェクトに挑戦してください！")